In [ ]:
seed_value= 0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)


import tensorflow as tf
#tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)

from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain 

from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers

In [ ]:
df=pd.read_csv("training.csv")
#df=pd.read_csv("validation.csv")  #Validation-set
#df=pd.read_csv("testing.csv")  #test-set

# ***Labelling***

In [ ]:
df['result']=0


In [ ]:
def labelling(data):
    windowsize = 14
    counterrow = 0
    numberofdays = data.shape[0]
    result = np.array(data['result'])
    while(counterrow < numberofdays):
        counterrow = counterrow + 1
        if(counterrow > windowsize):
            windowbeginindex = counterrow - windowsize
            windowendindex = windowbeginindex + windowsize - 1
            windowmiddleindex = (windowbeginindex + windowendindex)/2
            minimum = max(data['Adj Close'])
            maximum = 0
            for i in range(windowbeginindex-1, windowendindex):               
                number = data['Adj Close'].iloc[i]
                if(number < minimum):
                    minimum = number
                    minindex = i
                if(number > maximum):
                    maximum = number
                    maxindex = i
            
            result[minindex] = 1    ## 1 for buy  ,  0 for hold
            result[maxindex] = 2    ## 2 for sell
    return result

In [ ]:
df["result"]=labelling(df)


# ***Normalization***

In [ ]:
df=df.fillna(-12345)


In [ ]:
for n in range(6,21):
    df['rsi' + str(n)] = talib.RSI(df['Adj Close'].values, timeperiod=n)
    df['roc' + str(n)] = talib.ROC(df['Adj Close'].values, timeperiod=n)
    df['sma' + str(n)] = talib.SMA(df['Adj Close'].values, timeperiod=n)
    df['ema' + str(n)] = talib.EMA(df['Adj Close'].values, timeperiod=n)
    df['wma' + str(n)] = talib.WMA(df['Adj Close'].values, timeperiod=n)
    df['tema' + str(n)] = talib.TEMA(df['Adj Close'].values, timeperiod=n)
    df['william' + str(n)] = talib.WILLR(df['High'].values,df['Low'].values,df['Adj Close'].values, timeperiod=n)
    df['cci' + str(n)] = talib.CCI(df['High'].values,df['Low'].values,df['Adj Close'].values, timeperiod=n)
    df['cmo' + str(n)] = talib.CMO(df['Adj Close'].values, timeperiod=n)
    df['macd' + str(n)],df['macdSignal' + str(n)] ,df['macdHist' + str(n)]  = talib.MACD(df['Adj Close'].values,fastperiod=12, slowperiod=26, signalperiod=9)
    df['ppo' + str(n)] = talib.PPO(df['Adj Close'].values, fastperiod=12, slowperiod=26, matype=0)
    df['parabolicsar'+ str(n)] = talib.SAR(df['High'].values, df['Low'].values, acceleration=0, maximum=0)

In [ ]:
rsi=df[['rsi'+str(n) for n in range(6,21)]].to_numpy()
roc=df[['roc'+str(n) for n in range(6,21)]].to_numpy()
sma=df[['sma'+str(n) for n in range(6,21)]].to_numpy()
ema=df[['ema'+str(n) for n in range(6,21)]].to_numpy()
wma=df[['wma'+str(n) for n in range(6,21)]].to_numpy()
tema=df[['tema'+str(n) for n in range(6,21)]].to_numpy()
william=df[['william'+str(n) for n in range(6,21)]].to_numpy()
cci=df[['cci'+str(n) for n in range(6,21)]].to_numpy()
cmo=df[['cmo'+str(n) for n in range(6,21)]].to_numpy()
macd=df[['macd'+str(n) for n in range(6,21)]].to_numpy()
ppo=df[['ppo'+str(n) for n in range(6,21)]].to_numpy()
parabolicsar=df[['parabolicsar'+str(n) for n in range(6,21)]].to_numpy()



In [ ]:
train_images=[[j for j in chain.from_iterable([rsi[i],roc[i],sma[i],ema[i],william[i],cci[i],cmo[i],macd[i],ppo[i],tema[i],wma[i],parabolicsar[i]])]for i in df.index]


In [ ]:
train_images=np.array(train_images)


train_images=train_images.reshape(1423*180)


In [ ]:
temp=np.sort(train_images)
temp[::-1].sort()


In [ ]:
def normalization(data,x):
 
 unique = np.unique(x, axis=0)
 unique =unique.tolist()
 a=len(unique)
 for n in range(6,21):
  for i in range(0,1732):
    data.at[i,'rsi'+str(n)]=unique.index(data['rsi'+str(n)][i])/a
    data.at[i,'roc'+str(n)]=unique.index(data['roc'+str(n)][i])/a
    data.at[i,'sma'+str(n)]=unique.index(data['sma'+str(n)][i])/a
    data.at[i,'ema'+str(n)]=unique.index(data['ema'+str(n)][i])/a
    data.at[i,'wma' +str(n)]=unique.index(data['wma'+str(n)][i])/a
    data.at[i,'tema'+str(n)]=unique.index(data['tema'+str(n)][i])/a
    data.at[i,'william'+str(n)]=unique.index(data['william'+str(n)][i])/a
    data.at[i,'cci'+str(n)]=unique.index(data['cci'+str(n)][i])/a
    data.at[i,'cmo'+str(n)]=unique.index(data['cmo'+str(n)][i])/a
    data.at[i,'macd'+str(n)]=unique.index(data['macd'+str(n)][i])/a
    data.at[i,'ppo'+str(n)]=unique.index(data['ppo'+str(n)][i])/a
    data.at[i,'parabolicsar'+str(n)]=unique.index(data['parabolicsar'+str(n)][i])/a
 return data
  

df =normalization(df,temp)


# ***Saving file for further use***

In [ ]:
df.to_csv('training.csv')